In [ ]:
# %%
import optuna
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.impute import KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import tensorflow as tf
from keras import layers, models, regularizers, callbacks
from keras.api.optimizers import Adam
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt


# %%
# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load the dataset
df = pd.read_csv("./lucas_pre.csv")

# Display basic information
print(f"Dataset shape: {df.shape}")

# Identify column types
target = "pH_H2O"
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if target in numeric_cols:
    numeric_cols.remove(target)

categorical_cols = ["Depth", "LC", "LU", "USDA", "ISSS", "NUTS_0", "LC0_Desc", "LC1_Desc", "LU1_Desc"]
# Filter to only keep categorical columns that exist in the dataset
categorical_cols = [col for col in categorical_cols if col in df.columns]

# Remove columns with too many unique values or too many missing values
filtered_cat_cols = []
for col in categorical_cols:
    if col in df.columns:
        if df[col].nunique() < 30 and df[col].isna().sum() / len(df) < 0.3:
            filtered_cat_cols.append(col)

print(f"Using {len(numeric_cols)} numeric columns and {len(filtered_cat_cols)} categorical columns")

# Drop rows where target is missing
df = df.dropna(subset=[target])

# Feature engineering - create new features
if "EC" in df.columns and "OC" in df.columns:
    df["EC_OC_ratio"] = df["EC"] / df["OC"].replace(0, 0.001)

if "Clay" in df.columns and "Sand" in df.columns:
    df["Clay_Sand_ratio"] = df["Clay"] / df["Sand"].replace(0, 0.001)

if "N" in df.columns and "P" in df.columns and "K" in df.columns:
    # NPK balance is important for soil chemistry
    if df["N"].notnull().sum() > 0 and df["P"].notnull().sum() > 0 and df["K"].notnull().sum() > 0:
        df["NPK_sum"] = df["N"] + df["P"] + df["K"]

if "CaCO3" in df.columns:
    # Soil pH is strongly related to CaCO3
    df["log_CaCO3"] = np.log1p(df["CaCO3"])

if "OC" in df.columns and "N" in df.columns:
    # C:N ratio is important for soil biology
    df["CN_ratio"] = df["OC"] / df["N"].replace(0, 0.001)

if "Clay" in df.columns and "OC" in df.columns:
    # Clay-organic matter interactions affect pH
    df["Clay_OC_interaction"] = df["Clay"] * df["OC"]

# Extract target values
y = df[target].values

# Update numeric columns after adding engineered features
numeric_cols = [col for col in df.select_dtypes(include=[np.number]).columns.tolist() if col != target]

# Create preprocessing pipeline
numeric_transformer = Pipeline(
    steps=[
        ("imputer", KNNImputer(n_neighbors=5)),
        ("power", PowerTransformer(method="yeo-johnson")),  # Better than StandardScaler for skewed data
        ("scaler", StandardScaler()),
    ]
)

categorical_transformer = Pipeline(steps=[("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))])

# Combine all transformers
preprocessor = ColumnTransformer(
    transformers=[("num", numeric_transformer, numeric_cols), ("cat", categorical_transformer, filtered_cat_cols)]
)

# Prepare features
X = df[numeric_cols + filtered_cat_cols]

# Split data before preprocessing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# Apply preprocessing
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

print(f"Processed feature shape: {X_train_processed.shape}")


# %%
# Define a function to create the neural network model
def create_model(
    input_dim,
    neurons_layers=[128, 64, 32],
    dropout_rates=[0.4, 0.3, 0.2],
    activation="relu",
    learning_rate=0.001,
    l2_reg=0.001,
):
    model = models.Sequential()

    # Input layer
    model.add(layers.Input(shape=(input_dim,)))

    # Hidden layers
    for i, neurons in enumerate(neurons_layers):
        model.add(
            layers.Dense(
                neurons,
                activation=activation,
                kernel_regularizer=regularizers.l2(l2_reg),
                kernel_initializer="he_normal",
            )
        )
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(dropout_rates[i]))

    # Output layer
    model.add(layers.Dense(1))

    # Compile
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss="mse", metrics=["mae"])

    return model


# %%
# Define the objective function for Optuna
def objective(trial):
    # Hyperparameters to optimize
    n_layers = trial.suggest_int("n_layers", 1, 5)
    neurons_layers = []
    dropout_rates = []

    # Define ranges for each layer's neurons and dropout
    for i in range(n_layers):
        neurons_layers.append(trial.suggest_int(f"n_units_l{i}", 16, 512))
        dropout_rates.append(trial.suggest_float(f"dropout_l{i}", 0.1, 0.5))

    # Other hyperparameters
    activation = trial.suggest_categorical("activation", ["relu", "elu", "selu"])
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    l2_reg = trial.suggest_float("l2_reg", 1e-6, 1e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64, 128])

    # Create model with the suggested hyperparameters
    model = create_model(
        input_dim=X_train_processed.shape[1],
        neurons_layers=neurons_layers,
        dropout_rates=dropout_rates,
        activation=activation,
        learning_rate=learning_rate,
        l2_reg=l2_reg,
    )

    # Define callbacks
    early_stopping = callbacks.EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True, verbose=0)
    reduce_lr = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=5, min_lr=0.00001, verbose=0)

    # Train the model
    history = model.fit(
        X_train_processed,
        y_train,
        validation_split=0.2,
        epochs=100,  # Reduced max epochs for optimization
        batch_size=batch_size,
        callbacks=[early_stopping, reduce_lr],
        verbose=0,  # Silent training for optimization
    )

    # Get validation MAE from the best epoch
    val_mae = min(history.history["val_mae"])

    return val_mae  # Return validation MAE to minimize


# %%
# Run Optuna optimization
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)  # Adjust number of trials as needed

# %%
# Print optimization results
print("Number of finished trials:", len(study.trials))
print("Best trial:")
trial = study.best_trial

print(f"  Value (Validation MAE): {trial.value:.4f}")
print("  Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# %%
# Create and train the model with the best hyperparameters
best_n_layers = trial.params["n_layers"]
best_neurons = [trial.params[f"n_units_l{i}"] for i in range(best_n_layers)]
best_dropouts = [trial.params[f"dropout_l{i}"] for i in range(best_n_layers)]

best_model = create_model(
    input_dim=X_train_processed.shape[1],
    neurons_layers=best_neurons,
    dropout_rates=best_dropouts,
    activation=trial.params["activation"],
    learning_rate=trial.params["learning_rate"],
    l2_reg=trial.params["l2_reg"],
)

# Print best model summary
best_model.summary()

# %%
# Train the best model with the optimal hyperparameters
early_stopping = callbacks.EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True, verbose=1)
reduce_lr = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=5, min_lr=0.00001, verbose=1)

history = best_model.fit(
    X_train_processed,
    y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=trial.params["batch_size"],
    callbacks=[early_stopping, reduce_lr],
    verbose=1,
)

# %%
# Evaluate the best model
loss, mae = best_model.evaluate(X_test_processed, y_test)
print(f"Test Mean Absolute Error: {mae:.4f}")

# Predict on test data
y_pred = best_model.predict(X_test_processed)
r2 = r2_score(y_test, y_pred)
print(f"R² Score: {r2:.4f}")

# %%
# Plot training history
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Model Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Train", "Validation"], loc="upper right")

plt.subplot(1, 2, 2)
plt.plot(history.history["mae"])
plt.plot(history.history["val_mae"])
plt.title("Model MAE")
plt.ylabel("MAE")
plt.xlabel("Epoch")
plt.legend(["Train", "Validation"], loc="upper right")
plt.tight_layout()
plt.show()

# %%
# Visualize optimization process
plt.figure(figsize=(10, 6))
optuna.visualization.matplotlib.plot_optimization_history(study)
plt.tight_layout()
plt.show()

# %%
# Plot parameter importances
plt.figure(figsize=(10, 6))
optuna.visualization.matplotlib.plot_param_importances(study)
plt.tight_layout()
plt.show()

# %%
# Plot parallel coordinate plot for hyperparameters
plt.figure(figsize=(12, 8))
optuna.visualization.matplotlib.plot_parallel_coordinate(study)
plt.tight_layout()
plt.show()

# Plot predictions vs actual
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], "r--")
plt.xlabel("Actual pH")
plt.ylabel("Predicted pH")
plt.title(f"Actual vs Predicted pH (R² = {r2:.4f})")
plt.grid(True)
plt.tight_layout()
plt.show()
